In [1]:
import sys
import os
os.chdir("..")

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'src')))

In [2]:
import pandas as pd
from src.modeling import preprocess_data
from src.modeling import train_models
from src.evaluation import evaluate_models

# Load data
X_train, X_test, y_train, y_test = preprocess_data.load_modeling_data()

In [4]:
# Train models
rf_model = train_models.train_random_forest(X_train, y_train)
xgb_model = train_models.train_xgboost(X_train, y_train)
lgb_model = train_models.train_lightgbm(X_train, y_train)

# For MLP, we need to scale X_train first
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

mlp_model = train_models.train_mlp(pd.DataFrame(X_train_scaled), y_train)

# Compare models
models = {
    "Random Forest": rf_model,
    "XGBoost": xgb_model,
    "LightGBM": lgb_model,
    "MLP": mlp_model
}

# Evaluate
results_df = evaluate_models.compare_models(models, X_test, y_test)
display(results_df)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000217 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1489
[LightGBM] [Info] Number of data points in the train set: 3866, number of used features: 6
[LightGBM] [Info] Start training from score 7.029097


C:\Users\ASUAS\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but MLPRegressor was fitted without feature names
  warnings.warn(


,MAE,RMSE,R2
Model,,,
Random Forest,0.673644,0.911084,5.325429e-01
XGBoost,0.690668,0.928647,5.143470e-01
LightGBM,0.669790,0.909999,5.336558e-01
MLP,4485.678437,7121.409362,-2.855992e+07
